Este colab fue desarrollado por Arnold Charry Armero.

# Producción de Escritorios y Mesas

Usted puede producir escritorios y mesas consumiendo tres recursos. Usted debe de tener en cuenta que para producir un escritorio se necesitan 3 unidades de madera y para producir una mesa se necesitan 5 unidades de madera. Para producir un escritorio se necesita una hora y para producir una mesa se necesitan 2 horas. Para producir un escritorio en la máquina se necesitan 50 minutos y para producir una mesa en la máquina se necesitan 20 minutos. En total, se tienen 3600 unidades de madera, 1600 horas laborales y 48000 minutos. El precio de venta es de 700 para los escritorios y 900 para las mesas. Maximice los ingresos de la compañía.

#Conjuntos

$ i = \text{Productos indexados por } i$

$ j = \text{Recursos indexadas por } j$

## Parámetros

$ I_{i} = \textrm{Ingreso del producto }i $

$ C_{ij} = \textrm{Consumo del recurso } j \textrm{ para el producto } i$

$ R_{j} = \textrm{Cantidad de recurso máximo } j$

## Variable de decisión

$ x_{i} = \textrm{Cantidad a producir del producto } i $

## Función Objetivo

$$ \textrm{Max } z = \sum_{i = 1}^{n} I_{i}\: x_{i} $$

## Restricciones

No se puede rebasar el recurso $j$,

$$ \sum_{i = 1}^{n} x_{i} C_{ij} \leq R_{j}, \forall j$$

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 112.8 MB/s eta 0:00:00


In [2]:
from gurobipy import *

In [3]:
# Conjuntos
productos = range(2)
recursos = range(3)

In [4]:
# Parámetros
I = [700, 900]

C = [[3, 5],
    [1, 2],
    [50, 20]]

R = [3600, 1600, 48000]

In [5]:
# Crear la instancia del modelo
model = Model("Asignacion_Recursos")

# Inicialización de las variables de decisión
X = model.addVars(productos, vtype=GRB.CONTINUOUS, name="X")

# Agregar la función objetivo
model.setObjective(quicksum(I[i] * X[i] for i in productos), GRB.MAXIMIZE)

# Agregar las restricciones

for j in recursos:
    model.addConstr(quicksum(C[j][i] * X[i] for i in productos) <= R[j], name=f"Recurso_{j}")

# Resolver el problema
model.optimize()

# Imprimir el estado del problema
if model.Status == GRB.OPTIMAL:
    print("Status: Optimal")

# Resultados
for var in model.getVars():
  print(var.VarName, "=", var.X)

print(f"\nFunción Objetivo = {model.objVal:.2f}")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xfc16a329
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8947368e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.894736842e+05
Status: Optimal
X[0] = 884.2105263157895
X[1] = 189.47368421052633

Función Objetivo = 789473.68


## Precios Sombra

In [6]:
# Conjuntos
productos = range(2)
recursos = range(3)

In [7]:
# Parámetros
I = [700, 900]

C = [[3, 5],
    [1, 2],
    [50, 20]]

R = [3600, 1600, 48000]

In [8]:
# Crear el modelo
model = Model("Asignacion_Recursos")

# Variables
X = model.addVars(productos, vtype=GRB.CONTINUOUS, name="X")

# Función objetivo
model.setObjective(quicksum(I[i] * X[i] for i in productos), GRB.MAXIMIZE)

# Restricciones
restricciones = {}
for j in recursos:
    restricciones[j] = model.addConstr(quicksum(C[j][i] * X[i] for i in productos) <= R[j],
                                       name=f"Recurso_{j}")

# Resolver
model.optimize()

# Resultados
if model.Status == GRB.OPTIMAL:
    print("\n=== Resultados ===")
    for var in model.getVars():
        print(f"{var.VarName} = {var.X:.2f}")

    print(f"\nFunción Objetivo = {model.objVal:.2f}")

    print("\n=== Precios Sombra (valores duales) ===")
    for j in recursos:
        print(f"{restricciones[j].ConstrName}: {restricciones[j].Pi:.2f}")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xfc16a329
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8947368e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.894736842e+05

=== Resultados ===
X[0] = 884.21
X[1] = 189.47

Función Objetivo = 789473.68

=== Precios Sombra (valores duales) ===
Recurso_0: 163.16
Recurso_1: 0.00
Recurso_2: 4.21


In [10]:
def linear_example(limitations_list):

  # Conjuntos
  productos = range(2)
  recursos = range(3)

  # Parámetros
  I = [700, 900]

  C = [[3, 5],
    [1, 2],
    [50, 20]]

  R = limitations_list

  # Crear la instancia del modelo
  model = Model("Asignacion_Recursos")

  # Inicialización de las variables de decisión
  X = model.addVars(productos, vtype=GRB.CONTINUOUS, name="X")

  # Agregar la función objetivo
  model.setObjective(quicksum(I[i] * X[i] for i in productos), GRB.MAXIMIZE)

  # Agregar las restricciones

  for j in recursos:
    model.addConstr(quicksum(C[j][i] * X[i] for i in productos) <= R[j], name=f"Recurso_{j}")

  # Resolver el problema
  model.optimize()

  # Imprimir el estado del problema
  if model.Status == GRB.OPTIMAL:
    print("Status: Optimal")

  # Resultados
  for var in model.getVars():
    print(var.VarName, "=", var.X)

  print(f"\nFunción Objetivo = {model.objVal:.2f}")

  return model

In [11]:
limitations_list = [3600, 1600, 48000]
origin_model1 = linear_example(limitations_list)

# Usar Pi para obtener el precio sombra de cada restricción
print('El precio sombre de la restricción de madera es {shadow_price}.'.format(shadow_price = origin_model1.Pi[0]))
print('El precio sombre de la restricción de horas laborales es {shadow_price}.'.format(shadow_price = origin_model1.Pi[1]))
print('El precio sombre de la restricción de horas de máquina es {shadow_price}.'.format(shadow_price = origin_model1.Pi[2]))

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xfc16a329
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8947368e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.894736842e+05
Status: Optimal
X[0] = 884.2105263157895
X[1] = 189.47368421052633

Función Objetivo = 789473.68
El precio sombre de la restricción de madera es 163.1578947368421.
El prec